In [40]:
import boto3
from mypy_boto3_elasticbeanstalk import ElasticBeanstalkClient
from pprint import pprint


In [41]:
elasticbeanstalk_client: ElasticBeanstalkClient = boto3.client(
    "elasticbeanstalk", region_name="us-east-1"
)


In [42]:
elasticbeanstalk_client.list_platform_versions()


{'PlatformSummaryList': [{'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/(BETA) Node.js 12 running on 64bit Amazon Linux 2/0.1.0',
   'PlatformOwner': 'AWSElasticBeanstalk',
   'PlatformStatus': 'Ready',
   'PlatformCategory': '(BETA) Node.js',
   'OperatingSystemName': 'Amazon Linux',
   'OperatingSystemVersion': '2',
   'SupportedTierList': ['WebServer/Standard', 'Worker/SQS/HTTP'],
   'SupportedAddonList': ['Log/CloudWatch',
    'Log/S3',
    'Monitoring/Healthd',
    'WorkerDaemon/SQSD'],
   'PlatformVersion': '0.1.0',
   'PlatformBranchName': '(BETA) Node.js 12 running on 64bit Amazon Linux 2',
   'PlatformBranchLifecycleState': 'Retired'},
  {'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/.NET Core running on 64bit Amazon Linux 2/2.2.5',
   'PlatformOwner': 'AWSElasticBeanstalk',
   'PlatformStatus': 'Ready',
   'PlatformCategory': '.NET Core',
   'OperatingSystemName': 'Amazon Linux',
   'OperatingSystemVersion': '2',
   'SupportedTierList': ['WebServ

In [43]:
application_name: str = "assignment4"
template_name: str = "assignment4-tmp"
environment_name: str = "assignment4-env"
platform_arn: str = "arn:aws:elasticbeanstalk:us-east-1::platform/Node.js 14 running on 64bit Amazon Linux 2/5.4.5"
bucket_name: str = "ass4"
bucket_key: str = "application.zip"


In [44]:
try:
    application = elasticbeanstalk_client.create_application(
        ApplicationName=application_name
    )
except Exception as e:
    print("Already created!")


In [45]:
try:
    template = elasticbeanstalk_client.create_configuration_template(
        ApplicationName=application_name,
        TemplateName=template_name,
        PlatformArn= platform_arn
    )

    template
except Exception as e:
    print("Already created!")

In [46]:
try:
    application_version = elasticbeanstalk_client.create_application_version(
        ApplicationName=application_name,
        VersionLabel="1.0",
        SourceBundle={"S3Bucket": bucket_name, "S3Key": bucket_key},
    )

    application_version

except Exception as e:
    print(e)


In [47]:
# See https://docs.aws.amazon.com/elasticbeanstalk/latest/dg/environments-create-api.html#:~:text=sampleapplication-,OptionSettings.member.1.Namespace,aws-elasticbeanstalk-ec2-role,-Example

environment = elasticbeanstalk_client.create_environment(
    ApplicationName=application_name,
    EnvironmentName=environment_name,
    TemplateName=template_name,
    PlatformArn=platform_arn,
    OptionSettings=[
        {
            "Namespace": "aws:autoscaling:launchconfiguration",
            "OptionName": "IamInstanceProfile",
            "Value": "aws-elasticbeanstalk-ec2-role",
        },
        {
            "Namespace": "aws:autoscaling:asg",
            "OptionName": "MinSize",
            "Value": "1",
        },
        {
            "Namespace": "aws:autoscaling:asg",
            "OptionName": "MaxSize",
            "ResourceName": "ScheduledScaleUpSpecificTime",
            "Value": "3",
        },
        {
            "Namespace": "aws:autoscaling:scheduledaction",
            "OptionName": "MaxSize",
            "ResourceName": "ScheduledScaleUpSpecificTime",
            "Value": "3",
        },
        {
            "Namespace": "aws:autoscaling:scheduledaction",
            "OptionName": "MinSize",
            "ResourceName": "ScheduledScaleUpSpecificTime",
            "Value": "1",
        },
        {
            "Namespace": "aws:autoscaling:scheduledaction",
            "OptionName": "DesiredCapacity",
            "ResourceName": "ScheduledScaleUpSpecificTime",
            "Value": "1",
        },
    ],
)

environment


{'EnvironmentName': 'assignment4-env',
 'EnvironmentId': 'e-kmcwemxmf3',
 'ApplicationName': 'assignment4',
 'SolutionStackName': '64bit Amazon Linux 2 v5.4.5 running Node.js 14',
 'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/Node.js 14 running on 64bit Amazon Linux 2/5.4.5',
 'DateCreated': datetime.datetime(2021, 9, 14, 18, 26, 32, 262000, tzinfo=tzutc()),
 'DateUpdated': datetime.datetime(2021, 9, 14, 18, 26, 32, 262000, tzinfo=tzutc()),
 'Status': 'Launching',
 'Health': 'Grey',
 'Tier': {'Name': 'WebServer', 'Type': 'Standard', 'Version': '1.0'},
 'EnvironmentArn': 'arn:aws:elasticbeanstalk:us-east-1:602291782684:environment/assignment4/assignment4-env',
 'ResponseMetadata': {'RequestId': 'c7e88115-4b0c-45e1-98f4-d03630d13a97',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'text/xml',
   'date': 'Tue, 14 Sep 2021 18:26:31 GMT',
   'x-amzn-requestid': 'c7e88115-4b0c-45e1-98f4-d03630d13a97',
   'content-length': '1056',
   'connection': 'keep-alive'},
  

In [48]:
elasticbeanstalk_client.describe_environments()["Environments"]

[{'EnvironmentName': 'assignment4-env',
  'EnvironmentId': 'e-kmcwemxmf3',
  'ApplicationName': 'assignment4',
  'SolutionStackName': '64bit Amazon Linux 2 v5.4.5 running Node.js 14',
  'PlatformArn': 'arn:aws:elasticbeanstalk:us-east-1::platform/Node.js 14 running on 64bit Amazon Linux 2/5.4.5',
  'DateCreated': datetime.datetime(2021, 9, 14, 18, 26, 32, 240000, tzinfo=tzutc()),
  'DateUpdated': datetime.datetime(2021, 9, 14, 18, 26, 32, 448000, tzinfo=tzutc()),
  'Status': 'Launching',
  'AbortableOperationInProgress': False,
  'Health': 'Grey',
  'HealthStatus': 'Pending',
  'Tier': {'Name': 'WebServer', 'Type': 'Standard', 'Version': '1.0'},
  'EnvironmentLinks': [],
  'EnvironmentArn': 'arn:aws:elasticbeanstalk:us-east-1:602291782684:environment/assignment4/assignment4-env'},
 {'EnvironmentName': 'assignment4-env',
  'EnvironmentId': 'e-mn2ftg43ke',
  'ApplicationName': 'assignment4',
  'SolutionStackName': '64bit Amazon Linux 2 v5.4.5 running Node.js 14',
  'PlatformArn': 'arn:aw

In [ ]:
cloudfront_client = boto3.client("cloudfront", region_name="us-east-1")

cloudfront_client.create_distribution(
    DistributionConfig={
        "DefaultCacheBehaviour": {
            "TargetOriginId": bucket_name,
            "ViewerProtocolPolicy": "allow-all",
        },
        "CallerReference": "cdn",
        "Enabled": True,
        "Comment": "distribution to host portfolio",
    }
)
